# MORDM

In [7]:
# Import general python packages
import pandas as pd
import numpy as np
import copy

# Import functions
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation
from problem_formulation import sum_over, sum_over_time, pick_over_time, stick_over_time, slick_over_time

# Loading in the necessary modules for EMA workbench and functions
from ema_workbench import (Model, MultiprocessingEvaluator, Scenario,
                           Constraint, ScalarOutcome, TimeSeriesOutcome, ArrayOutcome)
from ema_workbench.util import ema_logging
from ema_workbench import save_results, load_results
from ema_workbench.em_framework.optimization import (EpsilonProgress)

In [8]:
# Loading in all the 17 objectives via predefined problem formulation 3
if __name__ == '__main__':
    ema_logging.log_to_stderr(ema_logging.INFO)
    dike_model, planning_steps = get_model_for_problem_formulation(7)

In [9]:
# Replicate the objectives
for outcome in dike_model.outcomes:
    print(outcome)
    

ArrayOutcome('Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths', 'A.5_Expected Number of Deaths'), function=<function sum_over_time at 0x000001A89CB73920>)
ArrayOutcome('A.1_Expected Annual Damage')
ArrayOutcome('A.2_Expected Annual Damage')
ArrayOutcome('A.3_Expected Annual Damage')
ArrayOutcome('A.4_Expected Annual Damage')
ArrayOutcome('A.5_Expected Annual Damage')
ArrayOutcome('Total_period_Costs', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs', 'RfR Total Costs', 'Expected Evacuation Costs'), function=<function sum_over_time at 0x000001A89CB73920>)


In [10]:
values = dike_model.outcomes['Total_period_Costs'].variable_name
print(len(values))  # This will show you the number of elements

7


In [11]:
# with MultiprocessingEvaluator(dike_model) as evaluator:
#     results = evaluator.perform_experiments(scenarios= 1, policies=1)

In [12]:
# Replicate the objectives
for outcome in dike_model.outcomes:
    print(outcome.shape)

None
None
None
None
None
None
None


In [18]:
# Writing a function to create actor specific problem formulations
def problem_formulation_actor(problem_formulation_actor):
   
    # Load the model:
    function = DikeNetwork()
    # workbench model:
    model = Model('dikesnet', function=function)
    # Outcomes are all costs, thus they have to minimized:
    direction = ScalarOutcome.MINIMIZE
    
    model.uncertainties = uncertainties
    model.levers = levers
    
    cost_variables = []
    # cost_variables.extend(
    # [
    #     f"{dike}_{e}"
    #     for e in ["Expected Annual Damage", "Dike Investment Costs"]
    #     for dike in function.dikelist
    # ])
    # cost_variables.extend([f"RfR Total Costs"])
    # cost_variables.extend([f"Expected Evacuation Costs"])


    if problem_formulation_actor == 4: #RWS
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage',
                            variable_name=['{}_Expected Annual Damage'.format(dike)
                                                for dike in function.dikelist],
                            function=sum_over, kind=direction),

            ScalarOutcome('Total Investment Costs',
                            variable_name=['{}_Dike Investment Costs'.format(dike)
                                                for dike in function.dikelist] + ['RfR Total Costs'
                                                                                ] + ['Expected Evacuation Costs'],
                            function=sum_over, kind=direction),

            ScalarOutcome('Expected Number of Deaths',
                            variable_name=['{}_Expected Number of Deaths'.format(dike)
                                                for dike in function.dikelist],
                            function=sum_over, kind=direction)] 
    
    elif problem_formulation_actor == 5: # GELDERLAND
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage A1', variable_name='A.1_Expected Annual Damage', function = sum_over, kind=direction),
            ScalarOutcome('Expected Annual Damage A2', variable_name='A.2_Expected Annual Damage', function = sum_over, kind=direction),
            ScalarOutcome('Expected Annual Damage A3', variable_name='A.3_Expected Annual Damage', function = sum_over, kind=direction),
            ScalarOutcome('Expected Number of Deaths in A1', variable_name='A.1_Expected Number of Deaths',function = sum_over, kind=direction),
            ScalarOutcome('Expected Number of Deaths in A2', variable_name='A.2_Expected Number of Deaths',function = sum_over, kind=direction),
            ScalarOutcome('Expected Number of Deaths in A3', variable_name='A.3_Expected Number of Deaths',function = sum_over, kind=direction),
            ScalarOutcome('Total Costs', variable_name=cost_variables, function = sum_over, kind=direction),
            ScalarOutcome('Aggregated Expected Number of Deaths A4-A5', variable_name = 
            ['A.4_Expected Number of Deaths', 'A.5_Expected Number of Deaths'], function = sum_over, kind=direction)]
    
    elif problem_formulation_actor == 6: #OVERIJSSEL
        model.outcomes.clear()
        
        model.outcomes = [
            ScalarOutcome(f'Total_period_Costs_0', variable_name= dike_model.outcomes['Total_period_Costs'].variable_name, function = stick_over_time, kind=direction),
            ScalarOutcome(f'Total_period_Costs_1', variable_name= dike_model.outcomes['Total_period_Costs'].variable_name, function = slick_over_time, kind=direction)
        ]
        
    else:
        raise TypeError('unknown identifier')
        
    return model

In [19]:
# Replicate the uncertainties
uncertainties = dike_model.uncertainties
uncertainties = copy.deepcopy(dike_model.uncertainties)

In [20]:
levers = dike_model.levers 
levers = copy.deepcopy(dike_model.levers)

In [21]:
model = problem_formulation_actor(6)


In [22]:
for tr in model.outcomes:
    print(tr)

ScalarOutcome('Total_period_Costs_0', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs', 'RfR Total Costs', 'Expected Evacuation Costs'), function=<function stick_over_time at 0x000001A89CB88040>)
ScalarOutcome('Total_period_Costs_1', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs', 'RfR Total Costs', 'Expected Evacuation Costs'), function=<function slick_over_time at 0x000001A89CB880E0>)


In [23]:
reference_values = {
    "Bmax": 175,
    "Brate": 1.5,
    "pfail": 0.5,
    "ID flood wave shape": 4,
    "planning steps": 2,
}
reference_values.update({f"discount rate {n}": 3.5 for n in planning_steps})
refcase_scen = {}

for key in dike_model.uncertainties:
    name_split = key.name.split('_')
    if len(name_split) == 1:

        refcase_scen.update({key.name: reference_values[key.name]})
    else:
        refcase_scen.update({key.name: reference_values[name_split[1]]})
            
ref_scenario = Scenario('reference', **refcase_scen)

In [24]:
convergence_metrics = {EpsilonProgress()}
#constraint = [Constraint("Total Costs", outcome_names="Total Costs", function=lambda x: max(0, x - 700000000))]

results_df = pd.DataFrame()
with MultiprocessingEvaluator(model) as evaluator:
    for _ in range(1):
       results = evaluator.optimize(nfe=2, searchover='levers',
                                     convergence=convergence_metrics,
                                     epsilons=[1]*len(model.outcomes), reference=ref_scenario)
        
        
save_results(results, 'Week23_MORDM_Reference_1000_PD6.tar.gz')


[MainProcess/INFO] pool started with 4 workers
100it [00:51,  1.95it/s]                                                       
[MainProcess/INFO] optimization completed, found 7 solutions
[MainProcess/INFO] terminating pool


KeyError: 'epsilon_progress'

In [25]:
y,t=results

In [26]:
y

,0_RfR 0,0_RfR 1,0_RfR 2,0_RfR 3,0_RfR 4,1_RfR 0,1_RfR 1,1_RfR 2,1_RfR 3,1_RfR 4,...,A.4_DikeIncrease 2,A.4_DikeIncrease 3,A.4_DikeIncrease 4,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,A.5_DikeIncrease 3,A.5_DikeIncrease 4,Total_period_Costs_0,Total_period_Costs_1
0,0,0,0,1,1,0,0,1,0,1,...,6,9,9,0,5,10,10,4,1.978081e+08,3.420187e+08
1,0,0,0,1,0,1,0,1,0,0,...,0,2,0,10,2,0,5,1,5.406807e+08,2.342095e+08
2,0,1,1,0,0,0,0,1,1,1,...,5,1,9,1,5,8,7,8,1.733730e+08,4.166836e+08
3,1,0,0,1,0,1,0,0,0,0,...,8,0,2,4,0,1,2,1,7.595939e+08,1.992417e+08
4,0,1,1,1,0,0,1,1,1,1,...,9,0,8,0,9,6,7,9,1.457690e+08,5.502322e+08
5,1,0,0,0,1,0,0,0,1,0,...,10,1,4,10,6,0,2,7,7.025938e+08,2.228848e+08
6,0,0,1,1,0,0,0,1,0,1,...,2,10,0,0,0,6,6,9,2.788141e+08,2.455040e+08


In [ ]:
convergence_metrics = {EpsilonProgress()}
constraint = [Constraint("Total_costs_0", outcome_names="Total_costs_0", function=lambda x: max(0, x - 1000))]

results_df = pd.DataFrame()
with MultiprocessingEvaluator(model) as evaluator:
    for _ in range(1):
       yui = evaluator.optimize(nfe=3, searchover='levers',
                                     convergence=convergence_metrics,
                                     epsilons=[1]*len(model.outcomes), reference=ref_scenario)
        
        



In [ ]:
from ema_workbench.em_framework.optimization import epsilon_nondominated, to_problem

problem = to_problem(model, searchover="levers")
epsilons = [0.05] * len(model.outcomes)
merged_archives = epsilon_nondominated(results_df, epsilons, problem)

In [ ]:
(y1,t1) , (y2,t2) = results

In [ ]:
t2

In [ ]:
y = y1+y2
y

# Gelderland


In [ ]:
model = problem_formulation_actor(5)
for outcome in model.outcomes:
    print(repr((outcome)))

In [ ]:
convergence_metrics = {EpsilonProgress()}
constraint = [Constraint("Total Costs", outcome_names="Total Costs", function=lambda x: max(0, x - 700000000))]

results_df = pd.DataFrame()
with MultiprocessingEvaluator(model) as evaluator:
    for _ in range(2):
       results = evaluator.optimize(nfe=2, searchover='levers',
                                     convergence=convergence_metrics,
                                     epsilons=[1]*len(model.outcomes), reference=ref_scenario,
                                     constraints=constraint)
        
        
save_results(results, 'Week23_MORDM_Reference_1000_PD6.tar.gz')
